# Lecture 04: `blockMesh` in OpenFOAM

<hr style="border:2px solid gray"> </hr>

## Authors

Ajay B Harish$^+$, Sanjay Govindjee and Frank McKenna <br>
*SimCenter, University of California, Berkeley* <br>

$^+$**Corresponding author** <br>
*Web:* www.ajaybharish.com <br>
*Email:* ajaybh@berkeley.edu <br>

## How to cite
Harish, Ajay Bangalore; Govindjee, Sanjay; McKenna, Frank (2020) “CFD Notebooks (Beginner).” DesignSafe-CI. https://doi.org/10.17603/ds2-w2x6-nm09.


## Copyright

The CFD Notebooks are developed by the NHERI <a href="http://simcenter.designsafe-ci.org">SimCenter</a>. It is copyrighted to “The Regents of the University of California” and is licensed under the BSD 2-clause license. For more information, please check our webpage for information on <a href="https://nheri-simcenter.github.io/CFD-Notebooks/geninfo/copyright.html">licensing and distribution</a>.

## Goal of this lecture

- What are the parts of a `blockMeshDict` file?
- Detail about each aspect of the `blockMeshDict`?
- Multiple blocks in `blockMeshDict`
- Curved edges in `blockMeshDict`
- **Estimated time for completion:** 30 minutes

For any queries, write to us on our <a href="http://simcenter-messageboard.designsafe-ci.org/smf/index.php?board=11.0">message board</a>.

<b>If you would like our feedback, you can post your solutions for the exercises to the <a href="http://simcenter-messageboard.designsafe-ci.org/smf/index.php?board=11.0">message board</a></b>.

In [41]:
# Introductory video for lecture 04 (L04a)
from IPython.display import IFrame
display(IFrame(('https://www.youtube.com/embed/ncIfsEhgLtU?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

`blockMesh` is one of the most basic mesh generators in OpenFOAM. It relies on a single dictionary file `blockMeshDict`, usually placed inside the folder `system` folder. `blockMesh` is used to create hexahedral blocks out of the domain, each block containing 8 vertices for each corner of the hexahedron. The `blockMeshDict` file contains the keywords that will be discussed in this lecture.

<hr style="border:2px solid gray"> </hr>

## Step 1/5: Introduction to `blockMeshDict`

In this section, we will discuss the parts of the `blockMesh` dictionary that are used to construct the parameters required for meshing. 

In [42]:
# Introduction to blockMesh (L04b)
display(IFrame(('https://www.youtube.com/embed/z4XAb7sH1pw?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

The principle behind `blockMesh` is to decompose the overall geometry into a set of one or more three dimensional, hexahedral block(s). The `blockMesh` can automatically handle one or more blocks. The edges of these blocks can be straight or curved lines. The curved lines are represented using splines. The mesh is specified by the number of cells to be generated in each of the x-, y- and z- directions. 

![Single block for blockMeshDict](https://nheri-simcenter.github.io/CFD-Notebooks/CFDN_images/blockMesh.png)


As shown in the above figure, in general, a block is defined by eight vertices. Each block is represented in the `blockMesh` dictionary as

    hex(0 1 2 3 4 5 6 7)

where 0, 1, ..., 7 represent the nodal numbers. However, it is also possible to generate blocks with less than 8 vertices by collapsing one or more vertex on the other and then the block is as shown below.

![Folded block for blockMeshDict](https://nheri-simcenter.github.io/CFD-Notebooks/CFDN_images/FoldedBlock.png)

Such a block can be represented in the `blockMesh` dictionary as

    hex (0 1 2 3 4 5 5 4)

As evident from above, the vertices are numbered and written such that they follow the right-hand rule as shown in the figures. Note here that the node numbers start from `0`!

Before proceeding ahead, let us try to access the `blockMesh` dictionary of the cavity problem from the previous lecture.

### Exercise 4.1

Why not try to print out the contents of the `blockMesh` dictionary, before proceeding ahead?

**Clue:** The path to the dictionary is `examples/cavity/system/blockMesh`

In [39]:
f = open("examples/cavity/system/blockMeshDict", "r")
file_contents = f.read()
print(file_contents)

/*--------------------------------*- C++ -*----------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  8
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

convertToMeters 0.1;

vertices
(
    (0 0 0)
    (1 0 0)
    (1 1 0)
    (0 1 0)
    (0 0 0.1)
    (1 0 0.1)
    (1 1 0.1)
    (0 1 0.1)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) (20 20 1) simpleGrading (1 1 1)
);

edges
(
);

boundary
(
    movingWall
    {
        type wall;
        faces
        (
            (3 7 6 2)
        );
    }
    fixedWalls
    {
        type wall;
        faces
        (
          

<hr style="border:2px solid gray"> </hr>

## Step 2/5: Parts of a `blockMesh` file

In this section, we will look through the `blockMesh` dictionary file to discuss the primary keywords used in the dictionary. We will further on see how these parameters affect the mesh generation process. 

The main keywords include :
- `converToMeters`
- `vertices`
- `edges`
- `block`
- `boundary`
- `mergePatchPairs`

We will look at each of these keywods in detail as we go ahead. 

In [43]:
# Basics of a blockMesh(L04c)
display(IFrame(('https://www.youtube.com/embed/77fpEmktZ3g?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

Let us print out the `blockMesh` dictionary from the `cavity` example before embarking on our journey to decipher the contents

In [40]:
f = open("examples/cavity/system/blockMeshDict", "r")
file_contents = f.read()
print(file_contents)

/*--------------------------------*- C++ -*----------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  8
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

convertToMeters 0.1;

vertices
(
    (0 0 0)
    (1 0 0)
    (1 1 0)
    (0 1 0)
    (0 0 0.1)
    (1 0 0.1)
    (1 1 0.1)
    (0 1 0.1)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) (20 20 1) simpleGrading (1 1 1)
);

edges
(
);

boundary
(
    movingWall
    {
        type wall;
        faces
        (
            (3 7 6 2)
        );
    }
    fixedWalls
    {
        type wall;
        faces
        (
          

### `convertToMeters` keyword

The first important keyword is the `convertToMeters`. This keyword specifies a scaling factor by which all the coordinates are scaled. In other words, if the coordinate provided is $x = 1.4$ and the scaling factor is $0.001$, then this implies that the values provided in the dictionary files are in millimeters.

In the above example, the scaling factor is $0.1$ and thus all the coordinates will be multiplied by $0.1$.

### `vertices` keyword

The `vertices` specifies the co-ordinates of all vertices of the three-dimensional block. For example: the `vertices` keyword is followed by a listing of the coordinates as follows:

    (
    (0 0 0)
    (1 0 0)
    (1 1 0)
    (0 1 0)
    (0 0 0.1)
    (1 0 0.1)
    (1 1 0.1)
    (0 1 0.1)
    );

Note here that the vertex numbers are only specified as comments but will be referred to in the next section on specification of the block. The vertex numbers start from  *0  rather than  1*  (like in most other commercial tools). 

### `block` keyword

The `block` specifies the connectivity of the vertices of the three-dimensional block. Additionally, a geometry can be decomposed into multiple blocks. In this case, each block needs to be specified. This is analogous to the mesh connectivity in a Finite Element mesh. 

In addition to the specification of the block structure, this also includes meshing information related to number of cells in the $x-, y-, z-$ directions. In addition to uniform mesh, it also facilitates a graded meshes.

In our `cavity` example, we have the block specification given to be

    (
    hex (0 1 2 3 4 5 6 7) 
    (20 20 1) 
    simpleGrading (1 1 1)
    );

- The first line `hex (0 1 2 3 4 5 6 7)` provides the connectivity of the block. It specifies that a hex-block is comprised of the vertices 0 - 7 and connected in the give order. 

- The second line `(20 20 1)` implies that we have 20 cells in the x-direction (1st index), 20 cells in the y-direction (2nd index) and 1 cell in the z-direction (3rd index).

- The third line `simpleGrading (1 1 1)` specifies that the mesh is uniform in all the three directions. The numbers *1,1,1* are known as the cell expansion rations. The expansion ratio enables the mesh to be graded, or refined, in any specified directions. The number specifies the ratio of the size of the end cell to that of the start cell along the edge/direction. There are two possible gradings, namely:

    - `simpleGrading`: The simpleGrading used here specifies that each edge along the direction is scaled uniformly.
    
    - `edgeGrading`: In contrast to uniform grading, one can also specify how each edge needs to be scaled. In the earlier figure of the block, the edge numbers are provided. Using these edge numbers, the edge scaling can be specified as
    
    edgeGrading (1 1 1 1 2 2 2 2 3 3 3 3)

It is also possible to provide multi-grading for a block. We will discuss this further in the lectures in the intermediate module.

### Exercise 4.2

Try modifying the blockMesh dictionary in our cavity example. Consider that the dimension of the block is $2\times2\times0.2$.

**Clue**: Consider the following steps:
- Copy the example file to your data depot
- Modify the `blockMesh` dictionary
- Initialize the agave
- Start the agave job as earlier. Here, note that the files dictionaries are now in your user folder and not the published folder. So the input directory in the `jobdetails` will change to `"inputDirectory": "agave://designsafe.storage.default/username/folder-path-to-cavity-example"`
- Run with the start and end times being the same. You can change this in the `controlDict` dictionary file in the `system` folder (remember Lecture 01). This will ensure that the mesh is created but the simulation does not start.

<hr style="border:2px solid gray"> </hr>

## Step 3/5: Using mutiple blocks and patches

Two of the most important aspects in the `blockMesh` dictionary includes the definition of multiple blocks and patches for definition of boundary conditions. In this section, we will discuss the details. At present, we will not be discussing aspects related to face matching or merging and these will be discussed in the intermediate CFD-notebooks.

In [44]:
# Multiple blocks and patches (L04d)
display(IFrame(('https://www.youtube.com/embed/c2V9k6KG2so?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

### Using multiple blocks

Most often, the geometries are more complicated and would require one to use more than one block to mesh. In the next section, we will see a similar example where we need three separated blocks to simulate fluid flow over a curved surface. In such instances, multiple blocks can be given in the block keyword as

    hex(0 1 2 3 4 5 6 7)
    hex(0 1 2 3 8 9 10 11)
    hex(4 5 6 7 12 13 14 15)
    
Above, we are defining three separate blocks. While the case is simple above, there can be cases when the faces of the blocks overlap. We will discuss the cases where such merging is required in our advanced lecture series.

### patches!

In our previous lecture, we discussed the various boundary condition options available in OpenFOAM. We also saw that the boundary conditions were defined in the dictionary files in the `0` folder. Each boundary condition was defined in reference to a particular patch. But we never discussed about defining the patch itself. The patches are defined in the `blockMesh` dictionary.

As we discussed so far, the geometry is divided into hexahedral blocks as shown in the earlier figure. Each face of this hexahedral block is referred to as a `patch`. Each patch is given a particular name to be referred to in the other files (like in the dictionaries in the `0` folder). A patch can be of two types: 

- `type`: This implies a generic patch on which boundary conditions can be directly specified.

- `faces`: This is a list of faces that together make up the patch.

        faces
        (
        
            (0 4 7 3)
            (2 6 5 1)
            (1 5 4 0)
                       
        );

As we see one of the example for faces from our `cavity/system/blockMesh` dictionary. We can see that there are three quadrilaterals (or faces of the block) that comprise of a single patch. In our example, we have specified three patches known as:

- `FrontAndBack`
- `MovingWall`
- `FixedWall`

For each of these patches, the related faces are also specified. Finally, by default, all faces that are omitted are collected by openFOAM and assigned to an `empty patch`.

### Exercise 4.3

Try modifying the `blockMesh` dictionary in our `cavity` example. Instead of one single block, let us split the geometry into three blocks, each of dimension $1\times1\times0.2$. While the overall dimension has changed, the problem at hand, itself remains unchanged.

**Clue**: Consider the following steps:
- Copy the example file to your data depot
- Modify the `blockMesh` dictionary
- Initialize the agave
- Start the agave job as earlier. Here, note that the files dictionaries are now in your user folder and not the published folder. So the input directory in the `jobdetails` will change to `"inputDirectory": "agave://designsafe.storage.default/username/folder-path-to-cavity-example"`
- Run with the start and end times being the same. You can change this in the `controlDict` dictionary file in the *system* folder (remember Lecture 01). This will ensure that the mesh is created but the simulation does not start.

<hr style="border:2px solid gray"> </hr>

## Step 4/5: Curved edges using splines

In this last section in our beginner course of the CFD Notebook series, we will consider the case of `blockMesh` with a curved edge. 

In [45]:
## Curved meshes (L04e)
display(IFrame(('https://www.youtube.com/embed/Wjx9BA4fI8g?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

Let us start with our cavity problem and try to modify our geometry and mesh using the `blockMesh` dictionary. Consider the geometry, with one curve edge as shown in the figure below.

<div>
<img src="https://nheri-simcenter.github.io/CFD-Notebooks/CFDN_images/CurvedEdge.png" width="500"/>
</div>


As we see, the middle part of the bottom edge is a curve. If we were to ignore this curve and mesh using the standard procedure of `blockMesh`, this will result in an inaccurate mesh that does not account for the curvature of the surface. The resulting inaccurate mesh is as shown below:<br>

<div>
<img src="https://nheri-simcenter.github.io/CFD-Notebooks/CFDN_images/MeshNobump.png" width="500"/>
</div>


However, the above is still a good practice to ensure no errors in the topology. Further on, we can use the the `edges` keyword in the blockMesh file to define a curve / polyline. Alongside the two end points, which also form the vertices of the hexahedron, some intermediate points are defined. The total number of points should be sufficient to describe the curve with reasonable accuracy. Here, the interpolation type used in *polyLine*. This means that a straight line is used between each of the interpolation points. Alternatively, splines can also be employed. The resulting `edges` specification is as shown below:

    edges
    (
        polyLine 1 2
        (
                (0	0       0)
                (0.1	0.0309016994    0)
                (0.2	0.0587785252    0)
                (0.3	0.0809016994    0)
                (0.4	0.0951056516    0)
                (0.5	0.1     0)
                (0.6	0.0951056516    0)
                (0.7	0.0809016994    0)
                (0.8	0.0587785252    0)
                (0.9	0.0309016994    0)
                (1	0       0)
        )

        polyLine 9 10
        (
                (0	0       1)
                (0.1	0.0309016994    1)
                (0.2	0.0587785252    1)
                (0.3	0.0809016994    1)
                (0.4	0.0951056516    1)
                (0.5	0.1     1)
                (0.6	0.0951056516    1)
                (0.7	0.0809016994    1)
                (0.8	0.0587785252    1)
                (0.9	0.0309016994    1)
                (1	0       1)
        )
    );


Here, the term `polyLine` is related to the interpolation type, $1$ and $2$ are related to the vertex numbers, the coordinates inside refer to the intermediate points. Thus, the resulting mesh, with a curved edge is as shown below:

![Block with curve for blockMeshDict](https://nheri-simcenter.github.io/CFD-Notebooks/CFDN_images/MeshWithbump.png)

### Exercise 4.4

- In our exercise 4.3, we have converted a single block into three blocks in our `blockMesh` dictionary. Can you use this to convert the lower edge of the middle block into a sin curve? Choose amplitude appropriately. 
- How does the mesh vary, with the change in amplitude? At some point the mesh starts to get distorted. Can you identify this point? 

Post your solutions on our <a href="http://simcenter-messageboard.designsafe-ci.org/smf/index.php?board=11.0">message board</a> for our feedback. 

<hr style="border:2px solid gray"> </hr>

## Step 5/5: Recap and next step

Before we move ahead to the next lecture, let us take a brief recap of this lecture. In this lecture, we explored the structure of a `blockMesh` dictionary file which is located in the `system` or `constant/polymesh` folder. The aim was to understand the primary keywords involved in the mesh generation process using the `blockMesh` utility.

In the upcoming lectures in our intermediate series, we will explore a more advanced utility called the `snappyHexMesh` along with other important topics.

In [46]:
## Recap and next steps (L04f)
display(IFrame(('https://www.youtube.com/embed/WrZsinOk7tU?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

Are you interested in trying out some sample problems. Check out our documentation site at <a href = "https://nheri-simcenter.github.io/CFD-Notebooks/lectures/beginner.html">Beginner lectures</a>.

We hope you enjoyed the beginner lectures of the CFD Notebooks. Please do leave comments and feedback on our <a href="http://simcenter-messageboard.designsafe-ci.org/smf/index.php?board=11.0">message board</a>. Looking forward to the intermediate series of CFD Notebooks.

# References

1. CFD-Notebooks documentation [(Link)](https://nheri-simcenter.github.io/CFD-Notebooks)
2. OpenFOAM Wiki [(Link)](https://openfoamwiki.net/index.php/Main_Page)
3. Official OpenFOAM documentation [(Link)](https://openfoam.org/resources)
4. Documentation from CFD Direct [(Link)](https://cfd.direct/openfoam/documentation)